In [2]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import Sequential, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import optimizers

In [ ]:
def load_own_model():
  model = Sequential()
  model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
  model.add(layers.MaxPooling2D(3))

  model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
  model.add(layers.MaxPooling2D(3))

  model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
  model.add(layers.MaxPooling2D(3))

  model.add(layers.Flatten())
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(7, activation='softmax'))

  model.compile(loss = 'categorical_crossentropy', 
                optimizer = 'adam',
                metrics = 'accuracy')
  
  return model

In [ ]:
model = load_own_model()
history = model.fit(X_train, y_train,validation_data = (X_val, y_val),
                    batch_size = 32, epochs = 500,
                    callbacks = EarlyStopping(patience = 20,restore_best_weights = True),
                    verbose = 1)

In [ ]:
res_1 = model.evaluate(X_test, y_test)

In [4]:
def load_vgg16():
  model = VGG16(include_top=False,input_shape=X_train.shape[1:])
  return model
model_vgg16 = load_vgg16()
model_vgg16.summary()

In [5]:
def set_nontrainable_layers(model):
    model.trainable = False
    return model

In [6]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(3, activation='softmax')

    model = Sequential([base_model,
                        flattening_layer,
                        dense_layer,
                        prediction_layer])
    return model